In [1]:
debug = True

In [2]:
import cv2
import numpy as np
import random
import sys 
import re
# from matplotlib import pyplot as plt

# Linux commands:
# sudo apt update
# sudo apt install tesseract-ocr
# sudo apt install libtesseract-dev

# !pip install pillow
# !pip install pytesseract
# !pip install opencv-python

from PIL import Image
import pytesseract
import argparse
# import cv2
import os

ModuleNotFoundError: No module named 'pytesseract'

In [49]:
def resizeImageByWidth(img, width=856):
    currRatio = img.shape[0]/img.shape[1]
#     print('Shape: ', img.shape)
    height = int(width * currRatio)
    dim = (width, height)
#     print('Resized shape: ', dim)
    # resize image
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return resized

In [70]:
def showImg(img, title = str(round(random.random()*1000,0))):
    if debug:
        cv2.imshow(title, img)
        cv2.waitKey(0)
        cv2.destroyWindow(title)

In [18]:
def cropImage(image, offsetX, offsetY, width, height):
    cardNu_threshold = 10
    minX = offsetX
    minY = offsetY
    cardNuWidth = width
    cardNuHeight = height

    return image[minY - cardNu_threshold:minY + cardNuHeight + cardNu_threshold, minX -
                   cardNu_threshold:minX + cardNuWidth + cardNu_threshold]


In [57]:
def histogramEqualize(img):
    #-----Converting image to LAB Color model-----------------------------------
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    # cv2.imshow("lab",lab)

    #-----Splitting the LAB image to different channels-------------------------
    l, a, b = cv2.split(lab)
    # cv2.imshow('l_channel', l)
    # cv2.imshow('a_channel', a)
    # cv2.imshow('b_channel', b)

    #-----Applying CLAHE to L-channel-------------------------------------------
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    #     cv2.imshow('CLAHE output', cl)

    #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
    limg = cv2.merge((cl, a, b))
    #     cv2.imshow('limg', limg)

    #-----Converting image from LAB Color model to RGB model--------------------
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    #     cv2.imshow('final', final)

    #     cv2.waitKey(0)
    #     cv2.destroyAllWindows()
    return final

## Util functions for extract text

In [19]:
# remove background, just keep text color from image
# imput image instance
def keepColor(image, lower, upper):
    frame = image.copy()
    # Convert BGR to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # cv.imshow("hsv", hsv)
    # define range of blue color in HSV

    # Threshold the HSV image to get only blue colors
    res = cv2.inRange(hsv, lower, upper)
    # Bitwise-AND mask and original image
#     res = cv2.bitwise_and(frame,frame, mask= mask)
#     cv2.imshow('frame',frame)
#     cv2.imshow('mask',mask)
#     cv2.imshow('res',res)
#     cv2.waitKey(0) 
#     cv2.destroyAllWindows()
    
    
    return res
    

In [20]:
def extractText(image):
    
    # Window: Point tesseract execute file path here
#     pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

    # Write down temporary file to extract
    filename = "{}.png".format(os.getpid())
    cv2.imwrite(filename, image)

    # extract text from image using tesseract ocr
    text = pytesseract.image_to_string(Image.open(filename))

    # Remove temporary image
    os.remove(filename)

    return text

## 1. Get Id number function

In [71]:
# get Id number function
# key color = in range [50 - 80]
def getIdNumber(image, V_upper_color=70):
    #load image
    originImg = image.copy()
#     showImg(originImg, "Origin image")
    
    #Crop id number area
    cropped_img = cropImage(originImg, 400, 140, 350, 40)
#     showImg(cropped_img, "Cropped")

    #filter
    
    lower = np.array([0,0,0])
    upper = np.array([180, 255, V_upper_color]) # V in range (50 - 80) is fine

    filteredImg = keepColor(cropped_img, lower, upper)
#     showImg(filteredImg)
    
    #extract text
    res = extractText(filteredImg)
    
    #remove non digit characters
    res = re.sub("\\D", '', res)
    return res


In [22]:
# print(getIdNumber('./data/temp4.jpg', 70)) #key

## 2. Get fullname

In [25]:
# get Id number function
# key color = in range [50 - 80]
def getFullname(image, V_upper_color=70):
    #load image
    originImg = image.copy()
#     showImg(originImg, "Origin image")
    
    #Crop id number area
    cropped_line1 = cropImage(originImg, 360, 195, 500, 40)
    cropped_line2 = cropImage(originImg, 275, 245, 500, 40)
    
#     showImg(cropped_line1, "Cropped 1")
#     showImg(cropped_line2, "Cropped 2")

    #filter
    
    lower = np.array([0,0,0])
    upper = np.array([180, 255, V_upper_color]) # V in range (50 - 80) is fine

    filtered_1 = keepColor(cropped_line1, lower, upper)
    filtered_2 = keepColor(cropped_line2, lower, upper)
    
#     showImg(filtered_1, "filter 1")
#     showImg(filtered_2, "filter 2")
    
    #extract text
    line1 = extractText(filtered_1).strip()
    line1 = re.sub("!(a-zA-Z)", '', line1)
    
    line2 = extractText(filtered_2).strip()
    line2 = re.sub("!(a-zA-Z)", '', line2)
 
    res = ''
    if len(line1) > 0:
        res = line1 + ' '
    res += line2
    
    
    return res.strip()


In [26]:
# print(getFullname('./data/temp4.jpg', 70)) #key

## 2. Get DOB

In [79]:
# get Id number function
# key color = in range [30 - 80]
def getDateOfBirth(image, V_upper_color=50):
    originImg = image.copy()
    showImg(originImg, "Origin image")
    
    #Crop id number area
    cropped_img = cropImage(originImg, 400, 300, 360, 40)
    showImg(cropped_img, "Cropped")

    #filter
    
    lower = np.array([0,0,0])
    upper = np.array([180, 255, V_upper_color]) # V in range (50 - 80) is fine

    filteredImg = keepColor(cropped_img, lower, upper)
    showImg(filteredImg, 'filtered')
    
    #extract text
    res = extractText(filteredImg)
    
    #remove non digit characters or '-' character
#     res = re.sub("!(\\d|-)", '', res)
    return res.strip()


In [81]:
print(getDateOfBirth(cv2.imread('./data/temp4.jpg'), 70)) #key

12-03-1998


## 3. Get POB

In [29]:
# get Id number function
# key color = in range [30 - 80]
def getPlaceOfBirth(image, V_upper_color=60):

    originImg = image.copy()
    #     showImg(originImg, "Origin image")
    
    #Crop id number area
    cropped_img_line1 = cropImage(originImg, 450, 340, 400, 40)
    cropped_img_line2 = cropImage(originImg, 280, 390, 600, 40)
#     showImg(cropped_img_line1, "Cropped line1")

    #filter
    
    lower = np.array([0,0,0])
    upper = np.array([180, 255, V_upper_color]) # V in range (50 - 80) is fine

    filtered_line1 = keepColor(cropped_img_line1, lower, upper)
    filtered_line2 = keepColor(cropped_img_line2, lower, upper)
    
#     showImg(filteredImg, 'filtered')
    
    #extract text
    line1 = extractText(filtered_line1)
    line1 = re.sub("!(a-zA-Z0-9|,)", '', line1)
    
    line2 = extractText(filtered_line2)    
    line2 = re.sub("!(a-zA-Z0-9|,)", '', line2)
    
    res = ''
    if len(line1) > 0:
        res = line1 + ' '
    res += line2
    
    #line 2
    return res.strip()


In [31]:
print(getPlaceOfBirth(imread'./data/temp4.jpg'))

## 4. Get Current address

In [32]:
# get Id number function
# key color = in range [30 - 80]
def getCurrentAddress(image, V_upper_color=50):
    originImg = image.copy()
#     showImg(originImg, "Origin image")
    
    #Crop id number area
    cropped_img_line1 = cropImage(originImg, 540, 435, 300, 40)
    cropped_img_line2 = cropImage(originImg, 240, 480, 600, 40)
#     showImg(cropped_img_line1, "Cropped line1")
#     showImg(cropped_img_line2, "Cropped line2")

    #filter
    
    lower = np.array([0,0,0])
    upper = np.array([180, 255, V_upper_color]) # V in range (50 - 80) is fine

    filtered_line1 = keepColor(cropped_img_line1, lower, upper)
    filtered_line2 = keepColor(cropped_img_line2, lower, upper)
    
#     showImg(filtered_line1, "filtered line1")
#     showImg(filtered_line2, "filtered line2")
    
    #extract text
    line1 = extractText(filtered_line1)
    line1 = re.sub("!(a-zA-Z0-9|,)", '', line1)
    
    line2 = extractText(filtered_line2)    
    line2 = re.sub("!(a-zA-Z0-9|,)", '', line2)
    
    res = ''
    if len(line1) > 0:
        res = line1 + ' '
    res += line2
    
    #line 2
    return res


In [33]:
# print(getCurrentAddress('./data/temp4.jpg', 50))

## 5. Get offical country name

In [34]:
# get Id number function
# key color = in range [30 - 80]
def getOfficalCountryName(image, V_upper_color=85):
    originImg = image.copy()
#     showImg(originImg, "Origin image")
    
    #Crop id number area
    cropped_line1 = cropImage(originImg, 275, 20, 560, 32)
#     showImg(cropped_line1, "Cropped line 1")

    #filter
    
    lower = np.array([0,0,0])
    upper = np.array([180, 255, V_upper_color]) # V in range (50 - 80) is fine

    filtered_line1 = keepColor(cropped_line1, lower, upper)
    
#     showImg(filtered_line1, 'filtered line 1')
    
    #extract text
    line1 = extractText(filtered_line1)
#     line1 = re.sub("!(a-zA-Z0-9)", '', line1)
    return line1


In [35]:
# print(getOfficalCountryName('./data/temp4.jpg'))

In [36]:
# get Offical title function
# key color = in range [30 - 80]
def getOfficalTitle(image, V_upper_color=100):
    originImg = image.copy()
#     showImg(originImg, "Origin image")
    
    #Crop id number area
    cropped_line2 = cropImage(originImg, 385, 65, 420, 12)
#     showImg(cropped_line2, "Cropped line 2")

    #filter
    
    lower = np.array([0,0,0])
    upper = np.array([180, 255, V_upper_color]) # V in range (50 - 80) is fine

    filtered_line2 = keepColor(cropped_line2, lower, upper)
#     showImg(filtered_line2, 'filtered line 2')
    
    #extract text
    line2 = extractText(filtered_line2)
    line2 = re.sub("!(a-zA-Z0-9)", '', line2)
    return line2


In [37]:
# print(getOfficalTitle('./data/temp4.jpg'))

## 6. Get card name

In [38]:
# get Offical title function
# key color = in range [30 - 80]
def getCardName(image, V_upper_color=100):
    originImg = image.copy()
#     showImg(originImg, "Origin image")
    
    #Crop id number area
    cropped_line2 = cropImage(originImg, 300, 99, 500, 38)
#     showImg(cropped_line2, "Cropped line 2")

    hsv = cv2.cvtColor(cropped_line2, cv2.COLOR_BGR2HSV)

    #get RED range colors
    mask1 = cv2.inRange(hsv, (0, 70, 50), (10,255,255))
    mask2 = cv2.inRange(hsv, (170,70,50), (180,255,255))
    
#     showImg(mask1, 'mask1')
#     showImg(mask2, 'mask2')
    
    
    ## Merge the mask and crop the red regions
    mask = cv2.bitwise_or(mask1, mask2 )
#     showImg(mask, 'mask')
    
#     res = cv2.bitwise_and(cropped_line2 , cropped_line2, mask= mask)
#     res = cv2.cvtColor(res, cv2.COLOR_HSV2BGR)
#     showImg(res, 'filtered line 2')
    
    #extract text
    line2 = extractText(mask)
    line2 = re.sub("!(a-zA-Z0-9)", '', line2)
    return line2


In [39]:
# print(getCardName('./data/temp4.jpg'))

# Get ID card information

In [63]:
#get
def extractInfo(imagePath):
    image = resizeImageByWidth(cv2.imread(imagePath))
    image = histogramEqualize(image)
    res={}
    res['img'] = imagePath
    res["country_name"] = getOfficalCountryName(image)
    res["country_title"] = getOfficalTitle(image)
    res['card_name'] = getCardName(image)
    res['id_number'] = getIdNumber(image)
    res['fullname'] = getFullname(image)
    res['date_of_birth'] = getDateOfBirth(image)
    res['place_of_birth'] = getPlaceOfBirth(image)
    res['current_address'] = getCurrentAddress(image)
    return res

In [66]:
testImg =  resizeImageByWidth(cv2.imread('./data/temp4.jpg'))
testImg = histogramEqualize(testImg)

In [65]:
extractInfo('./data/temp4.jpg')

{'img': './data/temp4.jpg',
 'country_name': 'CONG HOA XA HOI CHU NGHIA VIET NAM',
 'country_title': 'Dac lap - Tu do - Hanh phic',
 'card_name': 'GIAY CHUNG MINH NHAN DAN',
 'id_number': '225682227',
 'fullname': 'TRAN QUANG HUNG',
 'date_of_birth': 'sys ze,',
 'place_of_birth': '_.. TX.Ninb Hoa, Khanh Hoa ...',
 'current_address': '_Té Dan Phé 15, P.Ninh Hiép, TX.Ninh Hoa, Khanh Hoa'}

In [60]:
extractInfo('./data/1_134.png')

{'img': './data/1_134.png',
 'country_name': 'Proved rec erenr cr waa me',
 'country_title': 'b> ced ea',
 'card_name': 'Por tars Cees WIRE RH Ae OE',
 'id_number': '729250703',
 'fullname': '',
 'date_of_birth': 'Ror Oe rAnh i)\n\na',
 'place_of_birth': 'Tho6n Pha Thanh Thon, Khon Bang, Kon Tum',
 'current_address': 'IRE OU Gm ects Ban Nong La, Tay Ninh'}

In [64]:
extractInfo('./data/temp2.jpg')

{'img': './data/temp2.jpg',
 'country_name': 'CONG HOA XA HOI CHU NGHIA VIET NAM',
 'country_title': 'BDéc lap - Ty do - Hanh phic',
 'card_name': 'VA eo CCE CELE LL.',
 'id_number': '194642855',
 'fullname': '',
 'date_of_birth': 'Vi aL',
 'place_of_birth': '',
 'current_address': 'einai Dieta\n\n_Thi x4 Ba Dén, Quang Binh'}